<a href="https://colab.research.google.com/github/8persy/algoritms_colab/blob/main/%2211_311_%D0%A1%D1%82%D0%B5%D0%BA%2C_%D0%BE%D1%87%D0%B5%D1%80%D0%B5%D0%B4%D1%8C%2C_%D0%B4%D0%B5%D0%BA_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стек, очередь, дек

## Стек

<img src="https://d585tldpucybw.cloudfront.net/sfimages/default-source/blogs/2020/2020-11/lifo.png" height="600px"/>

- Каждый элемент попадает "вверх" структуры
- Вынимать элементы можно только "сверху"

Таким образом, тот элемент что пришел раньше, будет "ниже" по сравнению с тем, что пришел позже (Last In, First Out)

**Какие методы должны быть у стека?**

- **push**, положить элемент в стек
- **pop**, достать элемент из стека
- **peek**, вернуть (но не достать) верхний элемент

Все методы выполняются за $ O(1) $

Можно реализовать ручками при помощи NodeItem\
[Документация python](https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks) настаивает на использовании списков как стека

### Реализация стека ручками

In [ ]:
from __future__ import annotations
import dataclasses

@dataclasses.dataclass
class NodeItem:
  data: int
  next: NodeItem | None

In [ ]:
class ClassicStack:
  def __init__(self):
    self.__head = None

  def peek(self):
    if self.__head is None:
      return None
    else:
      return self.__head.data

  def push(self, data: int):
    if self.__head is None:
      self.__head = NodeItem(data=data, next=None)
    else:
      new_elem = NodeItem(data=data, next=self.__head)
      self.__head = new_elem

  def pop(self):
    if self.__head is None:
      return None
    else:
      data = self.__head.data
      self.__head = self.__head.next
      return data

  def __repr__(self):
    return self.__head.__repr__()


In [ ]:
classic_stack = ClassicStack()

In [ ]:
classic_stack.push(12)

In [ ]:
classic_stack

NodeItem(data=12, next=None)

In [ ]:
classic_stack.peek()

12

In [ ]:
classic_stack.push(5)

In [ ]:
classic_stack

NodeItem(data=5, next=NodeItem(data=12, next=None))

In [ ]:
classic_stack.pop()

5

In [ ]:
classic_stack

NodeItem(data=12, next=None)

### Где стеки применяются

Необходимо перевернуть и применить последовательность
- Отмена действий в редакторах
- Вычисление математических выражений
- Парсинг синтаксиса


### Стек действий

Сделаем кастомный стек действий для работы (и отмены наших действий с CLI Python), а так же простой интерфейс для этого

#### Работа с CLI

**Что будем использовать в работе с CLI?**

**eval** для вычисления **выражений**

In [ ]:
1 + 4

5

In [ ]:
eval("1 + 6")

7

In [ ]:
b = 5
eval("b + 1")

6

**exec** для исполнения кода изменения переменных

In [ ]:
exec("b = 7")

b

7

In [ ]:
exec("b *= 2")

eval("b**2")

196

**globals()** для получения текущих переменных\
**del** для удаления переменных

In [ ]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'class ActionStack:\n  def __init__(self):\n    self.__stack = []\n\n  def push(self, elem):\n    self.__stack.append(elem)\n\n  def peek(self):\n    if len(self.__stack) == 0:\n      return None\n    return self.__stack[-1]\n  \n  def pop(self, globs):\n    if len(self.__stack) == 0:\n      return None\n    else:\n      last_elem = self.__stack[-1]\n      parts = last_elem.split("=")\n      var = parts[0]\n      exec(f"del {var}",globs)\n      self.__stack = self.__stack[:-1]\n      return last_elem\n  \n  def apply_actions(self, globs):\n    for elem in self.__stack[::-1]:\n      exec(elem, globs)',
  'action_stack = ActionStack()',
  'action_stack.apply_actions(globals())\n\ndef is_exec(cmd):\n  exec_parts = 

In [ ]:
_i21

'exec("b = 7")\n\nb'

In [ ]:
b

14

In [ ]:
del b

In [ ]:
b

NameError: name 'b' is not defined

#### exec и globals

In [ ]:
def make_b_4():
  exec("b = 4")

In [ ]:
make_b_4()

In [ ]:
b = 5

In [ ]:
make_b_4()

In [ ]:
b

5

**Почему так?**\
Потому что globals в ячейке make_b_4 и в ячейках ниже разные\
**Что делать?**\
Пробрасывать globals в функцию

In [ ]:
def make_b_4(globs):
  exec("b = 4", globs)

In [ ]:
b

5

In [ ]:
make_b_4(globals())

In [ ]:
b

4

#### Чекпоинт 1

**Класс стека действий**

- Стек хранит список строк исполняемого (exec) кода в порядке его ввода с input
```python
stack = [
  "del a",
  "arr[0] = 4"
  "a = 15"
  "arr = [1, 2, 3]"
]
```
- Должны быть реализованы все методы стека (pop, push, peek)
- Должен быть реализован метод применения всего кода стека в хронологическом порядке порядке

**Интерфейс**
- Программа запускается и ждет ввода информации
- Если пользователь ввел что-то из специальных команд, необходимо запустить соответствующую обработку
- Если в строке не содержится ничего их признаков исполняемого кода, выводится **eval** введенного текста
- Иначе - помещаем в стек код и выполняем его (не забываем про **globals**)

**Команды интерфейса**
- **\q** - выход из программы (break)
- **\l** - last command, вывести верхнюю команду из стека (peek)
- **\p** - pop, достать верхнюю команду из стека + применить обновленную историю действий

In [ ]:
class ActionStack:
  def __init__(self):
    self.__stack = []

  def push(self, elem):
    self.__stack.append(elem)

  def peek(self) -> str:
    if len(self.__stack) > 0:
      return self.__steck[-1]
    else:
      return None

  def pop(self) -> str | None:
    if len(self.__stack) > 0:
      self.__steck.pop()
      return self.__steck[-1]
    else:
      return None

  def apply_actions(self, globs):
    for elem in self.__stack:
      exec(elem, globs)

  def __repr__(self):
    return self.__stack.__repr__()

In [ ]:
action_stack = ActionStack()

In [ ]:
action_stack.apply_actions(globals())

def is_exec(cmd: str) -> bool:
  """
    True, если в cmd содержится "=" или "del"
    False иначе
  """
  return '=' in cmd or cmd.startswith('del')

while True:
  text = input()
  globs = globals()

  if text == "\q":
    break
  elif text == "\p":
    print(action_stack.pop())
    print(action_stack)
  elif text == "\l":
    print(action_stack.peek())
  elif is_exec(text):
    action_stack.push(text)
    exec(text, globs)
  else:
    try:
      eval(text)
    except NameError:
      print("Неверный запрос")

b = 4


KeyboardInterrupt: Interrupted by user

#### Проблемы со стеком

Нет ограничения на количество хранимых команд, можем "упасть" из-за ограничений по памяти

In [ ]:
problem_stack = ActionStack()

In [ ]:
counter = 1
try:
  while True:
    problem_stack.push(f"p = {counter}")
    counter += 1
except Exception:
  print("Никогда не будет выполнено :(")
  print(f"p = {counter}")

**Как ограничивать стек?**
- Реализовать логику "Если больше N элементов" - не добавляем больше
  - Либо теряем хронологический порядок
  - Либо добавление занимает $ O(N) $
    - Ищем нижний элемент и ручками выбрасываем его
- Использовать очередь вместо стека

## Очередь

<img width="600px" src="https://d585tldpucybw.cloudfront.net/sfimages/default-source/blogs/2020/2020-11/fifo.png"/>

Очередь хранит 2 ссылки:
- **tail** на конец очереди
- **head** на начало очереди

За счет этого операции добавления/извлечения аналогичны по времени стеку ($O(1)$)\
Интерфейс очереди аналогичен стеку (pop, push, peek)\
Для некоторых задач дополнительно реализуется peek для последнего элемента в очереди

Можно использовать список (list) как очередь, но для большого кол-ва элементов будет тормозить (см.ниже)

### Где применяются очереди

- Там же где стеки, но нужно ограничение на кол-во элементов
- При синхронизации данных с удаленным источником
  - Посмотрим когда будем изучать многопоточность

### Реализация очереди ручками


In [ ]:
import abc

class Queue(abc.ABC):
  @abc.abstractmethod
  def peek(self):
    pass

  @abc.abstractmethod
  def push(self, data: int):
    pass

  @abc.abstractmethod
  def pop(self):
    pass

In [ ]:
class CustomQueue(Queue):
  def __init__(self):
    self.__head = None
    self.__tail = None

  def peek(self):
    if self.__head is None:
      return None
    else:
      return self.__head.data

  def push(self, data: int):
    if self.__head is None:
      self.__tail = NodeItem(data=data, next=None)
      self.__head = self.__tail
    else:
      new_elem = NodeItem(data=data, next=None)
      self.__tail.next = new_elem
      self.__tail = new_elem

  def pop(self):
    if self.__head is None:
      self.__tail = None
      return None
    else:
      data = self.__head.data
      self.__head = self.__head.next
      return data

  def __repr__(self):
    return self.__head.__repr__()

In [ ]:
custom_queue = CustomQueue()

In [ ]:
custom_queue.push(5)
custom_queue.push(10)
custom_queue.push(15)

custom_queue

NodeItem(data=5, next=NodeItem(data=10, next=NodeItem(data=15, next=None)))

In [ ]:
custom_queue.pop()

5

In [ ]:
custom_queue

NodeItem(data=10, next=NodeItem(data=15, next=None))

In [ ]:
custom_queue.push(7)

custom_queue

NodeItem(data=10, next=NodeItem(data=15, next=NodeItem(data=7, next=None)))

### Чекпоинт 2

Реализовать очередь с ограничением по кол-ву элементов

In [ ]:
class CustomLengthQueue(Queue):
  def __init__(self, size):
    self.__size = size
    self.__length = 0
    self.__head = None
    self.__tail = self.__head

  def peek(self):
    if self.__head is None:
      return None
    else:
      return self.__head.data

  def push(self, data: int):
    if self.__head is None:
      self.__head = NodeItem(data=data, next=None)
      self.__tail = self.__head

    elif self.__lenght == self.__size:
      self.pop()
      new_el = NodeItem(data=data, next=None)
      self.__tail.next = new_el
      self.__tail = new_el

    elif self.__lenght < self.__size:
      new_el = NodeItem(data=data, next=None)
      self.__tail.next = new_el
      self.__tail = new_el

    self.__length += 1

  def pop(self):
    if self.__head is None:
      return None
    else:
      el = self.__head.data
      self.__head = self.__head.next
      self.__lenght -=1
      return el

  def __repr__(self):
    return self.__head.__repr__()

Реализовать очередь с ограничением длины через list

In [ ]:
class LengthListQueue(Queue):
  def __init__(self, size):
    self.__size = size
    self.__queue = []

  def peek(self):
    return self.__queue[0]

  def push(self, data: int):
    if len(self.__queue) == self.__size:
      self.pop()
    self.__queue.append(data)

  def pop(self):
    el = self.__queue[0]
    self.__queue = self.__queue[1:]
    return el
  def __repr__(self):
    raise NotImplementedError()

### Когда ручками работает быстрее

In [ ]:
QUEUE_SIZE = 100_000

custom_queue = CustomLengthQueue(QUEUE_SIZE)
list_queue = LengthListQueue(QUEUE_SIZE)

In [ ]:
import time

def measure_push_time(queue: Queue, elem: int):
  start_time = time.time()
  queue.push(elem)
  end_time = time.time()

  return end_time - start_time

In [ ]:
import numpy as np

elements = np.arange(0, QUEUE_SIZE + 2, dtype=int)

for elem in elements:

  custom_queue.push(elem)
  list_queue.push(elem)

  if elem > QUEUE_SIZE:
    custom_queue_time = measure_push_time(custom_queue, elem)
    list_queue_time = measure_push_time(list_queue, elem)

    print(custom_queue_time)
    print(list_queue_time)

    print(list_queue_time / custom_queue_time)

    break

1.049041748046875e-05
0.0004818439483642578
45.93181818181818


## Дек (бонус)

<img height="300px" src="https://neerc.ifmo.ru/wiki/images/thumb/7/73/Deque1.png/200px-Deque1.png"/>

По сути - двусторонняя очередь

- **push_front / pop_front** - добавить/удалить элемент с "головы"
- **push_back / pop_back** - добавить/удалить элемент с "хвоста"

Все операции занимают $O(1)$

Из коробки [есть](https://docs-python.ru/standart-library/modul-collections-python/klass-deque-modulja-collections/) деки

### Дек из коробки

In [ ]:
import collections

dq = collections.deque([1, 2, 3])

dq

deque([1, 2, 3])

In [ ]:
dq.append(4)

dq.appendleft(0)

dq

deque([0, 1, 2, 3, 4])

In [ ]:
dq.popleft()

0

In [ ]:
dq

deque([1, 2, 3, 4])

### Чекпоинт 3 (бонус)

Реализовать CustomDeque

- push_front / pop_front
- push_back / pop_back
- peek_back / peek_front

In [ ]:
from __future__ import annotations
import dataclasses

@dataclasses.dataclass
class DoubleNodeItem:
  data: int
  prev: NodeItem | None = dataclasses.field(repr=False)
  next: NodeItem | None

In [ ]:
class CustomDeque:
  def __init__(self):
    self.__head = None
    self.__tail = self.__head

  def peek_back(self):
    if self.__head is None:
      return None
    else:
      return self.__tail.data

  def peek_front(self):
    if self.__head is None:
      return None
    else:
      return self.__head.data

  def push_back(self, data: int):
    if self.__head is None:
      self.__head = DoubleNodeItem(data = data, next = None, prev = None)
      self.__tail = self.__head
    else:
      a = DoubleNodeItem(data = data, next = None, prev = self.__tail)
      self.__tail.next = a
      self.__tail = a


  def push_front(self, data: int):
    if self.__head is None:
      self.__head = DoubleNodeItem(data = data, next = None, prev = None)
      self.__tail = self.__head
    else:
      a = DoubleNodeItem(data = data, next = self.__head, prev = None)
      self.__head.prev = a
      self.__head = a

  def pop_back(self):
    if self.__head is None:
      return None
    else:
      a = self.__tail.data
      self.__tail = self.__tail.prev
      self.__tail.next = None
      return a



  def pop_front(self):
    if self.__head is None:
      return None
    else:
      a = self.__head.data
      self.__head = self.__head.next
      self.__head.prev = None
      return a

  def __repr__(self):
    return self.__head.__repr__()

In [ ]:
custom_deque = CustomDeque()

In [ ]:
custom_deque.push_front(1)

In [ ]:
custom_deque

DoubleNodeItem(data=1, next=None)

In [ ]:
custom_deque.push_front(2)

In [ ]:
custom_deque

DoubleNodeItem(data=2, next=DoubleNodeItem(data=1, next=None))

In [ ]:
custom_deque.pop_back()

1

In [ ]:
custom_deque

DoubleNodeItem(data=2, next=None)